In [1]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import StackingClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

* Data Preparations


In [2]:
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")


In [3]:
def transform(df):
    object_df = df.select_dtypes(include=['object'])
    object_columns = object_df.columns
    return pd.get_dummies(object_df, prefix=object_columns), object_columns

def connectData(df, transformed, columns, all_dummy_columns):
    return pd.concat([df.drop(columns, axis=1), transformed.reindex(columns=all_dummy_columns, fill_value=False)], axis=1)

def split(title, df):
    return df.drop(title, axis=1), df[title]

In [4]:
train_transformed_data, train_deserted_columns = transform(train_df.drop(["id", "CustomerId", "Surname"], axis=1))
test_transformed_data, test_deserted_columns = transform(test_df.drop(["id", "CustomerId", "Surname"], axis=1))
all_dummy_columns = set(train_transformed_data.columns).union(test_transformed_data.columns)

train_converted_df = connectData(train_df, train_transformed_data, train_deserted_columns, all_dummy_columns)
test_converted_df = connectData(test_df, test_transformed_data, test_deserted_columns, all_dummy_columns)

train_x, train_y = split("Exited", train_converted_df.drop(["id", "CustomerId", "Surname"], axis=1))
test_x = test_converted_df.drop(["id", "CustomerId", "Surname"], axis=1)

# Find The Best Params

In [5]:
from sklearn.model_selection import GridSearchCV
def findParams(param_try, estimator):
    grid_search = GridSearchCV(estimator=estimator, param_grid=param_try, cv=5)
    grid_search.fit(train_x, train_y)
    return grid_search.best_params_

In [8]:
rfParams = findParams({
    'max_depth': [20, 30],
}, estimator=RandomForestClassifier())



In [9]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

def findParams(param_try, estimator):
    grid_search = GridSearchCV(estimator=estimator, param_grid=param_try, cv=5)
    grid_search.fit(train_x, train_y)
    return grid_search.best_params_

# 設置 XGBoost 的超參數搜索範圍
xgParams = findParams({
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}, xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0))

print("Best Parameters for XGBoost:", xgParams)


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [19:29:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [19:29:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [19:29:52] WARNING: C:\buildkite-age

Best Parameters for XGBoost: {'max_depth': 3, 'n_estimators': 100}


In [11]:
from sklearn.ensemble import GradientBoostingClassifier

# 定義 Gradient Boosting 的超參數搜索範圍
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# 創建 Gradient Boosting 分類器
gb_classifier = GradientBoostingClassifier()

# 使用 GridSearchCV 尋找最佳超參數
gb_grid_search = GridSearchCV(gb_classifier, gb_param_grid, scoring='accuracy', cv=5)
gb_grid_search.fit(train_x, train_y)  # 假設有定義 X_train 和 y_train

# 印出最佳超參數
print("Best Parameters for Gradient Boosting:", gb_grid_search.best_params_)


Best Parameters for Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# 定義 lightGBM 的超參數搜索範圍
lgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# 創建 lightGBM 分類器，並啟用 GPU 支援
lgb_classifier = lgb.LGBMClassifier(device='gpu', objective='binary', gpu_device_id=0)

# 使用 GridSearchCV 尋找最佳超參數
lgb_grid_search = GridSearchCV(lgb_classifier, lgb_param_grid, scoring='accuracy', cv=5)
lgb_grid_search.fit(train_x, train_y)  # 假設有定義 X_train 和 y_train

# 印出最佳超參數
print("Best Parameters for LightGBM:", lgb_grid_search.best_params_)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# 將資料分成訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# 定義 SVM 分類器
svm_classifier = SVC(probability=True)

# 定義 SVM 的超參數搜索範圍
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# 使用 GridSearchCV 尋找最佳超參數，並設置 verbose 參數
svm_grid_search = GridSearchCV(svm_classifier, svm_param_grid, scoring='accuracy', cv=5, n_jobs=-1, verbose=2)
svm_grid_search.fit(X_train, y_train)

# 印出最佳超參數
print("Best Parameters for SVM:", svm_grid_search.best_params_)



In [11]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
# 創建 MLP 分類器，啟用 GPU 支援
mlp = MLPClassifier(hidden_layer_sizes=(70, 50), activation='relu', solver='adam', max_iter=1500, validation_fraction=.05, early_stopping=True, n_iter_no_change=10)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# 創建 SVM 分類器，啟用 GPU 支援
svm_classifier = SVC(probability=True, gamma='scale', C=1, kernel='rbf', cache_size=4000)

# 使用找到的最佳超參數初始化 LightGBM 分類器
best_lgb_params = {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
lgb_classifier = LGBMClassifier(device='gpu', objective='binary', gpu_device_id=0, **best_lgb_params)

# 使用找到的最佳超參數初始化 XGBoost 分類器
best_xg_params = {'max_depth': 3, 'n_estimators': 100}
xg_classifier = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, **best_xg_params)

# 使用找到的最佳超參數初始化 Gradient Boosting 分類器
best_gb_params = {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5}
gb_classifier = GradientBoostingClassifier(**best_gb_params, random_state=42)


stacking_model = StackingClassifier(
    estimators=[
        ("xg", xg_classifier),
        ("rf", RandomForestClassifier(**rfParams)),
        ("svm", svm_classifier),
        ("gb", gb_classifier),
        ("lgb", lgb_classifier)
    ],
    final_estimator=mlp,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
)

In [12]:
constraint_interval = pd.read_csv("../analysis/interval_found.csv")
interval_list = constraint_interval.values.flatten()

In [13]:
stacking_model.fit(train_x, train_y)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [08:05:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [08:05:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 34921, number of negative: 130113
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 864
[LightGBM] [Info] Number of data points in the train set: 165034, number of used features: 13
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (1.89 MB) transferred to GPU in 0.012590 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211599 -> initscore=-1.315315
[LightGBM] [Info] Start training from score -1.315315


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [13:49:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [13:49:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\s

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 863
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (1.51 MB) transferred to GPU in 0.003203 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
[LightGB

StackingClassifier(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimators=[('xg',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=0, grow_pol...
                               ('rf', RandomForestClassifier(max_depth=20)),
                               ('svm',
                                SVC(C=1, cache_size=4000, probability=True)),
                               ('gb',
                                GradientBoostingClassifier(max_depth=5,
                                                           random_state=42)),
                               ('lgb',
                                LGBMClassifier(device='gpu', gpu_device_id=0,
                                               max_depth=7, n_estimators=200,
                                               objective='binary'))],
                   final_estimator=MLPClassifier(early_stopping=True,
                                                 hidden_layer_sizes=(70, 50),
                                                 max_iter=1500,
                                                 validation_fraction=0.05))

In [14]:
machine_predicting_ans = stacking_model.predict_proba(test_x)[:, 1]

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [10:21:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [15]:
submitting = pd.read_csv("../sample_submission.csv")

In [16]:
submit_test = test_x.copy()
submit_test['id'] = test_df['id']
submit_test['Exited'] = machine_predicting_ans
final_df = submitting.drop("Exited", axis=1).merge(submit_test[["Exited", "id"]],on='id', how='left')

In [17]:
final_df.to_csv("deeperperceptron122.csv", index=False)